In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets


In [ ]:
#

def data_generator(n, true_ate = False):
  d = pd.DataFrame()
  # Cofounders
  d['w1']
